In [21]:
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher 
from spacy.tokens import Span 
import networkx as nx
from tqdm import tqdm


In [22]:
faq = pd.read_csv('/content/faq.csv')
faq.shape

(90, 3)

In [24]:
faq.head()

,question,answer,category
0,What if I need to know my correct CGPA?,CGPA is seen on the grade card. The CGPA count...,grade card
1,I have cleared my backlog but the result sheet...,GPA and CGPA are not mentioned on backlog gra...,grade card
2,"I have cleared my backlog, but the result on E...",Grade card shows first attempt GPA and CGPA o...,grade card
3,What do I do if I have not received my grade c...,Grade cards should be collected from the stude...,grade card
4,My name/Father's Name/Mother's name is incorre...,Name correction on grade cards/PDC needs to b...,Corrections on grade card /PDC


In [25]:
doc = nlp(faq['question'][3])
for tok in doc: 
  print(tok.text, '---', tok.dep_)

What --- dobj
do --- aux
I --- nsubj
do --- ROOT
if --- mark
I --- nsubj
have --- aux
not --- neg
received --- advcl
my --- poss
grade --- compound
card --- dobj
? --- punct
